<a href="https://colab.research.google.com/github/durml91/MMath-Project/blob/duo-branch/Image_Diffusion_(working)/Attention_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jaxlib==0.4.2+cuda11.cudnn82 -f  https://storage.googleapis.com/jax-releases/jax_cuda_releases.html # [cuda]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
    Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
      Successfully uninstalled jaxlib-0.3.25+cuda11.cudnn805


In [2]:
!pip install diffrax
!pip install equinox
!pip install einops
!pip install optax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 KB 9.6 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.4-py3-none-any.whl size=1403844 sha256=fc39d3411f42499ff2440ab261dc46bcf1abdb29b0478ab219eb2b8d8a3ae11f
  Stored in directory: /root/.cache/pip/wheels/5e/cd/9b/750eb95db5b18b776ae59f55ae22b91a01e3703f3fb07eaa13
Successfully built jax
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1
  Attempting uninstall: jax
    Found existing installation: jax 0.3.25
    Uninstalling jax-0.3.25:
      Successfully uninstalled jax-0.3.25
Looking in indexes: https://pypi.org

In [3]:
import array
import functools as ft
import gzip
import os
import struct
import urllib.request

import diffrax as dfx  # https://github.com/patrick-kidger/diffrax
import einops  # https://github.com/arogozhnikov/einops
import jax
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
import optax  # https://github.com/deepmind/optax

import equinox as eqx

In [4]:
import shutil
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import shutil

In [152]:
# class qkv(eqx.Module):
#     c_attn: eqx.nn.Linear


#     def __init__(self, key):
#         self.c_attn = eqx.nn.Linear(64, 3*64, key=key)

#     def __call__(self, x):
#         #x = einops.rearrange(x, "a b -> b a")
        
#         #T,C = x.shape
#         print("hi")
#         qkv = jax.vmap(self.c_attn)(x)
#         #qkv = qkv.transpose(1,0)
#         #print(qkv.shape)
#         q,k,v = jnp.array_split(qkv, 3, axis=-1)
#         print(q.shape,k.shape,v.shape)
#         # q = einops.rearrange(q, "a b->b a")
#         # k = einops.rearrange(k, "a b->b a")
#         # v = einops.rearrange(v, "a b->b a")

        
        
#         # #print(x.shape)
#         # W_q = self.wq(x)
#         # #print(W_q.shape)
#         # W_k = self.wk(x)
#         # W_v = self.wv(x)
#         # Q = jnp.matmul(x, W_q)
#         # K = jnp.matmul(x, W_k)
#         # V = jnp.matmul(x, W_v)

#         return q,k,v


# class Attblock(eqx.Module):
#     matrix: eqx.Module

#     c_proj: eqx.nn.Linear
    
#     att: eqx.nn.MultiheadAttention

#     def __init__(self, key):

#         self.att = eqx.nn.MultiheadAttention(7, query_size=64, key=key)
#         self.matrix = qkv(key=key)
#         self.c_proj = eqx.nn.Linear(64,64, key=key)

#     def __call__(self, x):
#         Q,K,V = self.matrix(x)
        
#         at=self.att(Q,K,V)
#         at = self.c_proj(at)
#         return at

In [130]:
# class qkv(eqx.Module):
#     c_attn: eqx.nn.Linear


#     def __init__(self, key):
#         self.c_attn = eqx.nn.Linear(66, 3*66, key=key)

#     def __call__(self, x):
#         x = einops.rearrange(x, "a b -> b a")
        
#         #T,C = x.shape

#         qkv = jax.vmap(self.c_attn)(x)
#         #qkv = qkv.transpose(1,0)
#         #print(qkv.shape)
#         q,k,v = jnp.array_split(qkv, 3, axis=-1)
#         print(q.shape,k.shape,v.shape)
        

        
        
#         # #print(x.shape)
#         # W_q = self.wq(x)
#         # #print(W_q.shape)
#         # W_k = self.wk(x)
#         # W_v = self.wv(x)
#         # Q = jnp.matmul(x, W_q)
#         # K = jnp.matmul(x, W_k)
#         # V = jnp.matmul(x, W_v)

#         return q,k,v


# class Attblock(eqx.Module):
#     matrix: eqx.Module

#     c_proj: eqx.nn.Linear
    
#     att: eqx.nn.MultiheadAttention

#     def __init__(self, key):

#         self.att = eqx.nn.MultiheadAttention(7, query_size=66, key=key)
#         self.matrix = qkv(key=key)
#         self.c_proj = eqx.nn.Linear(66,66, key=key)

#     def __call__(self, x):
#         Q,K,V = self.matrix(x)
        
#         at=self.att(Q,K,V)
#         at = self.c_proj(at)
#         return at



# class SinusoidalEmbedding(eqx.Module):
#     d:int = 28


#     def __init__(self, d):
#         self.d=d

#     def __call__(self, x):
#       half_dim = self.d//2
#       emb=jnp.log(10000) / (half_dim - 1)
#       emb = jnp.exp(jnp.arange(half_dim)* -emb)
#       emb = x[:,None] * emb[None,:]
#       emb = jnp.concatenate([jnp.sin(emb), jnp.cos(emb)], -1)
#       return emb


# class TimeEmbed(eqx.Module):
#     dim: int =28
#     embed: eqx.nn.Embedding
#     sin: eqx.Module
    
#     def __init__(self, key):
#         self.embed = eqx.nn.Embedding(1000, 28, key=key)
#         self.sin = SinusoidalEmbedding(28)
  
#     def __call__(self, x):
#         se = self.sin(x)
#         se = (se*10000).astype(int)
#         emb = self.embed(se)
#         return emb




# class MixerBlock(eqx.Module):
#     patch_mixer: eqx.nn.MLP
#     hidden_mixer: eqx.nn.MLP
#     norm1: eqx.nn.LayerNorm
#     norm2: eqx.nn.LayerNorm

#     Att1: eqx.Module
#     Att2: eqx.Module
#     gnorm1: eqx.nn.GroupNorm
#     gnorm2: eqx.nn.GroupNorm


#     def __init__(
#         self, num_patches, hidden_size, mix_patch_size, mix_hidden_size, *, key
#     ):
#         tkey, ckey,a1key, a2key = jr.split(key, 4)
#         self.patch_mixer = eqx.nn.MLP(
#             num_patches, num_patches, mix_patch_size, depth=1, key=tkey
#         )
#         self.hidden_mixer = eqx.nn.MLP(
#             hidden_size, hidden_size, mix_hidden_size, depth=1, key=ckey
#         )
#         self.norm1 = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.norm2 = eqx.nn.LayerNorm((num_patches, hidden_size))

#         self.Att1 = Attblock(key = a1key)
#         self.Att2 = Attblock(key = a2key)

#         self.gnorm1 = eqx.nn.GroupNorm(4,32)
#         self.gnorm2 = eqx.nn.GroupNorm(4,32)

#     def __call__(self, y):
#         y = y + jax.vmap(self.patch_mixer)(self.norm1(y))
#         unatt = self.Att1(y)
#         gnormatt = self.gnorm1(unatt)
#         y = y + gnormatt
        
#         y = einops.rearrange(y, "c p -> p c")
#         y = y + jax.vmap(self.hidden_mixer)(self.norm2(y))
#         y = einops.rearrange(y, "p c -> c p")
        
#         unatt = self.Att2(y)
#         gnormatt = self.gnorm2(unatt)
#         y = y + gnormatt
#         return y




# class Mixer2d(eqx.Module):
#     conv_in: eqx.nn.Conv2d
#     conv_out: eqx.nn.ConvTranspose2d
#     blocks: list
#     norm: eqx.nn.LayerNorm
#     t1: float
#     timemb: eqx.Module


#     def __init__(
#         self,
#         img_size,
#         patch_size,
#         hidden_size,
#         mix_patch_size,
#         mix_hidden_size,
#         num_blocks,
#         t1,
#         *,
#         key,
#     ):
#         input_size, height, width = img_size
#         assert (height % patch_size) == 0
#         assert (width % patch_size) == 0
#         num_patches = (height // patch_size) * (width // patch_size)
        
#         inkey, outkey,tkey, *bkeys = jr.split(key, 3 + num_blocks)

#         self.conv_in = eqx.nn.Conv2d(
#             input_size + 1, hidden_size, patch_size, stride=patch_size, key=inkey
#         )
#         self.conv_out = eqx.nn.ConvTranspose2d(
#             hidden_size, input_size, patch_size, stride=patch_size, key=outkey
#         )
#         self.blocks = [
#             MixerBlock(
#                 num_patches, hidden_size, mix_patch_size, mix_hidden_size, key=bkey
#             )
#             for bkey in bkeys
#         ]
#         self.norm = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.t1 = t1
#         self.timemb = TimeEmbed(key=tkey)

#     def __call__(self, t, y):
#         t = t / self.t1
#         t = einops.repeat(t, "-> 1")
#         t = jnp.array([t], dtype=int)
#         t = self.timemb(t)
#         t = einops.rearrange(t, " c 1 h w-> c h w")
#         y = jnp.concatenate([y, t])
#         y = self.conv_in(y)
#         _, patch_height, patch_width = y.shape
#         y = einops.rearrange(y, "c h w -> c (h w)")
#         for block in self.blocks:
#             y = block(y)
#         y = self.norm(y)
#         y = einops.rearrange(y, "c (h w) -> c h w", h=patch_height, w=patch_width)
#         return self.conv_out(y)

In [34]:
# class qkv(eqx.Module):
#     wk: eqx.nn.Linear
#     wq: eqx.nn.Linear
#     wv: eqx.nn.Linear

#     def __init__(self, key):
#         self.wq = eqx.nn.Linear(49, 128, key=key)
#         self.wk = eqx.nn.Linear(49, 128, key=key)
#         self.wv = eqx.nn.Linear(49, 128, key=key)

#     def __call__(self, x):
#         W_q = jax.vmap(self.wq)(x)
#         W_k = jax.vmap(self.wk)(x)
#         W_v = jax.vmap(self.wv)(x)
        
#         print(W_q.shape, x.shape)
#         Q = jax.lax.batch_matmul(x, W_q)
#         K = jax.lax.batch_matmul(x, W_k)
#         V = jax.lax.batch_matmul(x, W_v)
#         return Q,K,V


# class Attblock(eqx.Module):
#     matrix: eqx.Module
    
#     att: eqx.nn.MultiheadAttention

#     def __init__(self, key):

#         self.att = eqx.nn.MultiheadAttention(7, query_size=49, key=key)
#         self.matrix = qkv(key=key)

#     def __call__(self, x):
#         Q,K,V = self.matrix(x)
        
#         at=self.att(Q,K,V)
#         return at




# class MixerBlock(eqx.Module):
#     patch_mixer: eqx.nn.MLP
#     hidden_mixer: eqx.nn.MLP
#     norm1: eqx.nn.LayerNorm
#     norm2: eqx.nn.LayerNorm

#     Att1: eqx.Module
#     Att2: eqx.Module
#     gnorm1: eqx.nn.GroupNorm
#     gnorm2: eqx.nn.GroupNorm


#     def __init__(
#         self, num_patches, hidden_size, mix_patch_size, mix_hidden_size, *, key
#     ):
#         tkey, ckey,a1key, a2key = jr.split(key, 4)
#         self.patch_mixer = eqx.nn.MLP(
#             num_patches, num_patches, mix_patch_size, depth=1, key=tkey
#         )
#         self.hidden_mixer = eqx.nn.MLP(
#             hidden_size, hidden_size, mix_hidden_size, depth=1, key=ckey
#         )
#         self.norm1 = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.norm2 = eqx.nn.LayerNorm((num_patches, hidden_size))

#         self.Att1 = Attblock(key = a1key)
#         self.Att2 = Attblock(key = a2key)

#         self.gnorm1 = eqx.nn.GroupNorm(4,32)
#         self.gnorm2 = eqx.nn.GroupNorm(4,32)

#     def __call__(self, y):
#         y = y + jax.vmap(self.patch_mixer)(self.norm1(y))
#         unatt = self.Att1(y)
#         gnormatt = self.gnorm1(unatt)
#         y = y + gnormatt
        
#         y = einops.rearrange(y, "c p -> p c")
#         y = y + jax.vmap(self.hidden_mixer)(self.norm2(y))
#         y = einops.rearrange(y, "p c -> c p")
        
#         unatt = self.Att2(y)
#         gnormatt = self.gnorm2(unatt)
#         y = y + gnormatt
#         return y




# class Mixer2d(eqx.Module):
#     conv_in: eqx.nn.Conv2d
#     conv_out: eqx.nn.ConvTranspose2d
#     blocks: list
#     norm: eqx.nn.LayerNorm
#     t1: float
#     tim_emb: eqx.nn.Embedding


#     def __init__(
#         self,
#         img_size,
#         patch_size,
#         hidden_size,
#         mix_patch_size,
#         mix_hidden_size,
#         num_blocks,
#         t1,
#         *,
#         key,
#     ):
#         input_size, height, width = img_size
#         assert (height % patch_size) == 0
#         assert (width % patch_size) == 0
#         num_patches = (height // patch_size) * (width // patch_size)
        
#         inkey, outkey,tkey, *bkeys = jr.split(key, 3 + num_blocks)

#         self.conv_in = eqx.nn.Conv2d(
#             input_size + 1, hidden_size, patch_size, stride=patch_size, key=inkey
#         )
#         self.conv_out = eqx.nn.ConvTranspose2d(
#             hidden_size, input_size, patch_size, stride=patch_size, key=outkey
#         )
#         self.blocks = [
#             MixerBlock(
#                 num_patches, hidden_size, mix_patch_size, mix_hidden_size, key=bkey
#             )
#             for bkey in bkeys
#         ]
#         self.norm = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.t1 = t1
#         self.tim_emb = eqx.nn.Embedding(28, 28, key=tkey)

#     def __call__(self, t, y):
#         t = t / self.t1
#         t = einops.repeat(t, "-> 1")
#         t = jnp.array([t], dtype=int)
#         t = self.tim_emb(t)
        
#         #_, height, width = y.shape
#         t = einops.repeat(t, "c h w-> c (28 h) w")
#         y = jnp.concatenate([y, t])
#         y = self.conv_in(y)
#         _, patch_height, patch_width = y.shape
#         y = einops.rearrange(y, "c h w -> c (h w)")
#         for block in self.blocks:
#             y = block(y)
#         y = self.norm(y)
#         y = einops.rearrange(y, "c (h w) -> c h w", h=patch_height, w=patch_width)
#         return self.conv_out(y)

In [4]:
from typing import Callable

In [5]:
key = jr.PRNGKey(2023)

In [95]:
#Dit

#Embedding Layer of Timesteps

class Lambda1(eqx.Module):
    fn: Callable

    def __call__(self , x, *, key=None):
        return self.fn(x)



class TimeStepEmbedder(eqx.Module):
    mlp: eqx.nn.Sequential
    frequency_embedding_size: int
    
    
    def __init__(
        self,
        hidden_size,
        frequency_embedding_size,   #set as 256
        key
    ):

        self.mlp = eqx.nn.Sequential([
            eqx.nn.Linear(frequency_embedding_size, hidden_size, key=key),
            Lambda1(jax.nn.silu),
            eqx.nn.Linear(hidden_size, hidden_size, key=key)
        ])
        self.frequency_embedding_size = frequency_embedding_size


    def __call__(
        self,
        t,
        max_period=10000
    ):
        dim = self.frequency_embedding_size
        half = dim // 2
        freqs = jnp.exp(
            -jnp.log(max_period) * jnp.arange(0, half, dtype=float) / half
        )
        args = t[:, None].astype(float) * freqs[None]
        
        embedding = jnp.concatenate([jnp.cos(args), jnp.sin(args)], axis=-1)
        print(embedding.shape)

        if dim % 2:
            embedding = jnp.concatenate([embedding, jnp.zeros_like(embedding[:, :1])], axis=-1)
        

        t_freq = embedding
        t_emb = jax.vmap(self.mlp)(t_freq)
        return t_emb      

In [96]:
#Time embed test
TimeStepEmbedder(32, 256, key)(jnp.array([4]))

(1, 256)


Array([[ 0.3052112 ,  0.19230524,  0.14764252,  0.31935525, -0.06504469,
        -0.23124412, -0.36936757,  0.02332152,  0.14454725, -0.18957621,
         0.20766026,  0.00606484,  0.03791146, -0.04720105, -0.12594688,
        -0.15844625, -0.09823439,  0.18520194, -0.12404346, -0.09638146,
        -0.04518805,  0.06181648, -0.0097344 , -0.07609019, -0.00168262,
         0.21857136, -0.17319855,  0.19038439, -0.12791844, -0.2365416 ,
         0.14090869,  0.08538287]], dtype=float32)

In [89]:
## attention

class MultiAtt(eqx.Module):

    c_attn: eqx.nn.Linear
    c_proj: eqx.nn.Linear

    n_head: int


    def __init__(
        self,
        n_embd,
        n_head,
        key
    ):

        assert n_embd % n_head ==0
        self.c_attn = eqx.nn.Linear(n_embd, 3 * n_embd, key=key)
        self.c_proj = eqx.nn.Linear(n_embd, n_embd, key=key)

        self.n_head = n_head

        #self.mha = eqx.nn.MultiheadAttention(n_head, )


    def __call__(
        self,
        x,
        mask=None
    ):
        B, T, C = x.shape

        qkv = jax.vmap(jax.vmap(self.c_attn))(x)
        qkv = qkv.reshape(B, T, self.n_head, -1)
        qkv = jnp.transpose(qkv, (0, 2, 1, 3)) #[batch, number of heads, T or seq_length, embed_dim or C(channels)]
        q, k, v = jnp.array_split(qkv, 3, axis=-1)

        

        new_k = jnp.swapaxes(k, -2,-1) *(1.0 / jnp.sqrt(k.shape[-1]))

        att = jax.lax.batch_matmul(q, new_k)

        if mask is not None:
            attn = jnp.where(mask == 0, -9e15, att)
        

        attention = jax.nn.softmax(att, axis=-1)


        values = jax.lax.batch_matmul(attention, v)


        values = jnp.transpose(values, (0,2,1,3)) # [batch, T, n_heads, C]
        values = values.reshape(B, T, C)

        o = jax.vmap(jax.vmap(self.c_proj))(values)



        return o, attention

In [42]:
x = jr.normal(key, (3, 16, 128))
print(x.shape)
mh_attn = MultiAtt(n_embd=128, n_head=4, key=key)
output, attention = mh_attn(x)
print(output.shape, attention.shape)

In [ ]:
## DitBlock

In [77]:
def single_loss_fn(model, weight, int_beta, data, t, key):
    mean = data * jnp.exp(-0.5 * int_beta(t))
    var = jnp.maximum(1 - jnp.exp(-int_beta(t)), 1e-5)
    std = jnp.sqrt(var)
    noise = jr.normal(key, data.shape)
    y = mean + std * noise
    pred = model(t, y)
    return weight(t) * jnp.mean((pred + noise / std) ** 2)


def batch_loss_fn(model, weight, int_beta, data, t1, key):
    batch_size = data.shape[0]
    tkey, losskey = jr.split(key)
    losskey = jr.split(losskey, batch_size)
    # Low-discrepancy sampling over t to reduce variance
    t = jr.uniform(tkey, (batch_size,), minval=0, maxval=t1 / batch_size)
    t = t + (t1 / batch_size) * jnp.arange(batch_size)
    loss_fn = ft.partial(single_loss_fn, model, weight, int_beta)
    loss_fn = jax.vmap(loss_fn)
    return jnp.mean(loss_fn(data, t, losskey))


@eqx.filter_jit
def single_sample_fn(model, int_beta, data_shape, dt0, t1, key):
    def drift(t, y, args):
        _, beta = jax.jvp(int_beta, (t,), (jnp.ones_like(t),))
        return -0.5 * beta * (y + model(t, y))

    term = dfx.ODETerm(drift)
    solver = dfx.Tsit5()
    t0 = 0
    y1 = jr.normal(key, data_shape)
    # reverse time, solve from t1 to t0
    sol = dfx.diffeqsolve(term, solver, t1, t0, -dt0, y1) #adjoint=dfx.NoAdjoint()
    return sol.ys[0]

In [78]:
def mnist():
    filename = "train-images-idx3-ubyte.gz"
    url_dir = "https://storage.googleapis.com/cvdf-datasets/mnist"
    target_dir = os.getcwd() + "/data/mnist"
    url = f"{url_dir}/{filename}"
    target = f"{target_dir}/{filename}"

    if not os.path.exists(target):
        os.makedirs(target_dir, exist_ok=True)
        urllib.request.urlretrieve(url, target)
        print(f"Downloaded {url} to {target}")

    with gzip.open(target, "rb") as fh:
        _, batch, rows, cols = struct.unpack(">IIII", fh.read(16))
        shape = (batch, 1, rows, cols)
        return jnp.array(array.array("B", fh.read()), dtype=jnp.uint8).reshape(shape)


def dataloader(data, batch_size, *, key):
    dataset_size = data.shape[0]
    indices = jnp.arange(dataset_size)
    while True:
        perm = jr.permutation(key, indices)
        (key,) = jr.split(key, 1)
        start = 0
        end = batch_size
        while end < dataset_size:
            batch_perm = perm[start:end]
            yield data[batch_perm]
            start = end
            end = start + batch_size
     

In [79]:
@eqx.filter_jit
def make_step(model, weight, int_beta, data, t1, key, opt_state, opt_update):
    loss_fn = eqx.filter_value_and_grad(batch_loss_fn)
    loss, grads = loss_fn(model, weight, int_beta, data, t1, key)
    updates, opt_state = opt_update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    key = jr.split(key, 1)[0]
    return loss, model, key, opt_state

In [128]:
def main(
    # Model hyperparameters
    patch_size=4,
    hidden_size=66,
    mix_patch_size=512,
    mix_hidden_size=512,
    num_blocks=4,
    t1=10.0,
    # Optimisation hyperparameters
    num_steps=500_000,
    lr=3e-4,
    batch_size=256,
    print_every=5_000,
    # Sampling hyperparameters
    dt0=0.1,
    sample_size=10,
    # Seed
    seed=2023,
):
    key = jr.PRNGKey(seed)
    model_key, train_key, loader_key, sample_key = jr.split(key, 4)
    data = mnist()
    data_mean = jnp.mean(data)
    data_std = jnp.std(data)
    data_max = jnp.max(data)
    data_min = jnp.min(data)
    data_shape = data.shape[1:]
    data = (data - data_mean) / data_std

    model = Mixer2d(
        data_shape,
        patch_size,
        hidden_size,
        mix_patch_size,
        mix_hidden_size,
        num_blocks,
        t1,
        key=model_key,
    )
    int_beta = lambda t: t  # Try experimenting with other options here!
    weight = lambda t: 1 - jnp.exp(
        -int_beta(t)
    )  # Just chosen to upweight the region near t=0.

    opt = optax.adabelief(lr)
    # Optax will update the floating-point JAX arrays in the model.
    opt_state = opt.init(eqx.filter(model, eqx.is_inexact_array))

    total_value = 0
    total_size = 0
    for step, data in zip(
        range(num_steps), dataloader(data, batch_size, key=loader_key)
    ):
        value, model, train_key, opt_state = make_step(
            model, weight, int_beta, data, t1, train_key, opt_state, opt.update
        )
        total_value += value.item()
        total_size += 1
        if (step % print_every) == 0 or step == num_steps - 1:
            print(f"Step={step} Loss={total_value / total_size}")
            total_value = 0
            total_size = 0

    
    eqx.tree_serialise_leaves("attmixer_mnist_model.eqx", model)
    shutil.copy('/content/attmixer_mnist_model.eqx','/content/gdrive/MyDrive/Colab_Notebooks')

    
    
    sample_key = jr.split(sample_key, sample_size**2)
    sample_fn = ft.partial(single_sample_fn, model, int_beta, data_shape, dt0, t1)
    sample = jax.vmap(sample_fn)(sample_key)
    sample = data_mean + data_std * sample
    sample = jnp.clip(sample, data_min, data_max)
    sample = einops.rearrange(
        sample, "(n1 n2) 1 h w -> (n1 h) (n2 w)", n1=sample_size, n2=sample_size
    )
    plt.imshow(sample, cmap="Greys")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
main()